Import appropriate libraries

In [4]:
import numpy as np
import pandas as pd

Read the html into dataframe

In [5]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = df[0]

display the dataframe

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop the rows that Borough is not assigned

In [7]:
df = df[~df['Borough'].str.contains('Not assigned')]

reset the index for later indexing

In [8]:
df.reset_index(inplace = True, drop = True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


find the index of unassigned Neighborhood

In [20]:
info = df[df['Neighbourhood'].isin(['Not assigned'])]
print(info.index)
info

Int64Index([], dtype='int64')


,Postcode,Borough,Neighbourhood


Make unassigned Neighbourhood equals to its Borough

In [10]:
df['Neighbourhood'][info.index] = df['Borough'][info.index]

Use groupby and aggregate function to assign Borough to each unique postcode and join all the Neighbourhood under the same postcode

In [11]:
df1 = df.groupby("Postcode").agg({"Borough": "unique","Neighbourhood":'，'.join}).reset_index()
df1

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],Rouge，Malvern
1,M1C,[Scarborough],Highland Creek，Rouge Hill，Port Union
2,M1E,[Scarborough],Guildwood，Morningside，West Hill
3,M1G,[Scarborough],Woburn
4,M1H,[Scarborough],Cedarbrae
5,M1J,[Scarborough],Scarborough Village
6,M1K,[Scarborough],East Birchmount Park，Ionview，Kennedy Park
7,M1L,[Scarborough],Clairlea，Golden Mile，Oakridge
8,M1M,[Scarborough],Cliffcrest，Cliffside，Scarborough Village West
9,M1N,[Scarborough],Birch Cliff，Cliffside West


In [12]:
df1.shape

(103, 3)

In [13]:
import sys
!{sys.executable} -m pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 13.5MB/s ta 0:00:01


Used Geospatial data povided intead of Geocoder due to its unreliablility in my region

In [21]:
geo = pd.read_csv("http://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


find the row position of my own Toronto dataframe postcode in the geospatial datafram

In [22]:
Latitude_Data = []
Longitude_Data = []
for code in df1['Postcode']:
    match = geo[geo['Postal Code'].isin([code])]
    Latitude_Data.append(geo["Latitude"][match.index[0]])
    Longitude_Data.append(geo["Longitude"][match.index[0]])
df1['Latitude'] = Latitude_Data
df1['Longitude'] = Longitude_Data


In [23]:
Neighbourhood_Data = []
Borough_Data = []
i = 0
while i <=102:
    Neighbourhood_Data.append(df1["Neighbourhood"][i])
    Borough_Data.append(df1["Borough"][i])
    i = i+1

In [24]:
df1

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,[Scarborough],Rouge，Malvern,43.806686,-79.194353
1,M1C,[Scarborough],Highland Creek，Rouge Hill，Port Union,43.784535,-79.160497
2,M1E,[Scarborough],Guildwood，Morningside，West Hill,43.763573,-79.188711
3,M1G,[Scarborough],Woburn,43.770992,-79.216917
4,M1H,[Scarborough],Cedarbrae,43.773136,-79.239476
5,M1J,[Scarborough],Scarborough Village,43.744734,-79.239476
6,M1K,[Scarborough],East Birchmount Park，Ionview，Kennedy Park,43.727929,-79.262029
7,M1L,[Scarborough],Clairlea，Golden Mile，Oakridge,43.711112,-79.284577
8,M1M,[Scarborough],Cliffcrest，Cliffside，Scarborough Village West,43.716316,-79.239476
9,M1N,[Scarborough],Birch Cliff，Cliffside West,43.692657,-79.264848


In [25]:
df1.shape

(103, 5)

In [26]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [29]:
map_toronto = folium.Map(location=[43.669542,-79.422564], zoom_start=12)
for Latitude_Data, Longitude_Data, Borough_Data, Neighbourhood_Data in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood_Data, Borough_Data)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude_Data, Longitude_Data],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

Work with only boroughs that contain the word Toronto

In [30]:
i = 0
filter_list = []
while i < 103:
    filter =  "Toronto" in df1['Borough'][i][0].split()
    i = i + 1
    filter_list.append(filter)
dfToronto = df1[filter_list].reset_index(drop = True)
dfToronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,[East Toronto],The Beaches,43.676357,-79.293031
1,M4K,[East Toronto],The Danforth West，Riverdale,43.679557,-79.352188
2,M4L,[East Toronto],The Beaches West，India Bazaar,43.668999,-79.315572
3,M4M,[East Toronto],Studio District,43.659526,-79.340923
4,M4N,[Central Toronto],Lawrence Park,43.728020,-79.388790
5,M4P,[Central Toronto],Davisville North,43.712751,-79.390197
6,M4R,[Central Toronto],North Toronto West,43.715383,-79.405678
7,M4S,[Central Toronto],Davisville,43.704324,-79.388790
8,M4T,[Central Toronto],Moore Park，Summerhill East,43.689574,-79.383160
9,M4V,[Central Toronto],Deer Park，Forest Hill SE，Rathnelly，South Hill，...,43.686412,-79.400049


In [31]:
map_toronto = folium.Map(location=[43.669542,-79.422564], zoom_start=12)
for Latitude_Data, Longitude_Data, Borough_Data, Neighbourhood_Data in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood_Data, Borough_Data)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude_Data, Longitude_Data],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

In [ ]:
map_toronto

In [32]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1OD5ZRUGTEFOXVLIGGVJVR3RMUBERTBI43UAEY1ZTKNAQT4K
CLIENT_SECRET:2CUCMLUDUOYKTYIHLEYNHHGLC0SCCNNFN2EF3EPLVK3TC5YP


In [33]:
dfToronto.loc[0, 'Neighbourhood']

'The Beaches'

In [34]:
neighborhood_latitude = dfToronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [35]:
LIMIT = 100

radius = 500



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1OD5ZRUGTEFOXVLIGGVJVR3RMUBERTBI43UAEY1ZTKNAQT4K&client_secret=2CUCMLUDUOYKTYIHLEYNHHGLC0SCCNNFN2EF3EPLVK3TC5YP&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [36]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc4eda4ad1789003915a945'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
Toronto_venues = getNearbyVenues(names=dfToronto['Neighbourhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )


The Beaches
The Danforth West，Riverdale
The Beaches West，India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park，Summerhill East
Deer Park，Forest Hill SE，Rathnelly，South Hill，Summerhill West
Rosedale
Cabbagetown，St. James Town
Church and Wellesley
Harbourfront，Regent Park
Ryerson，Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide，King，Richmond
Harbourfront East，Toronto Islands，Union Station
Design Exchange，Toronto Dominion Centre
Commerce Court，Victoria Hotel
Roselawn
Forest Hill North，Forest Hill West
The Annex，North Midtown，Yorkville
Harbord，University of Toronto
Chinatown，Grange Park，Kensington Market
CN Tower，Bathurst Quay，Island airport，Harbourfront West，King and Spadina，Railway Lands，South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place，Underground city
Christie
Dovercourt Village，Dufferin
Little Portugal，Trinity
Brockton，Exhibition Place，Parkdale Village
High Park，The Junction South
Parkdale，Roncesvall

In [40]:
Toronto_venues = getNearbyVenues(names=dfToronto['Neighbourhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )


The Beaches
The Danforth West，Riverdale
The Beaches West，India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park，Summerhill East
Deer Park，Forest Hill SE，Rathnelly，South Hill，Summerhill West
Rosedale
Cabbagetown，St. James Town
Church and Wellesley
Harbourfront，Regent Park
Ryerson，Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide，King，Richmond
Harbourfront East，Toronto Islands，Union Station
Design Exchange，Toronto Dominion Centre
Commerce Court，Victoria Hotel
Roselawn
Forest Hill North，Forest Hill West
The Annex，North Midtown，Yorkville
Harbord，University of Toronto
Chinatown，Grange Park，Kensington Market
CN Tower，Bathurst Quay，Island airport，Harbourfront West，King and Spadina，Railway Lands，South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place，Underground city
Christie
Dovercourt Village，Dufferin
Little Portugal，Trinity
Brockton，Exhibition Place，Parkdale Village
High Park，The Junction South
Parkdale，Roncesvall

In [42]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide，King，Richmond,100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
Brockton，Exhibition Place，Parkdale Village,22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
CN Tower，Bathurst Quay，Island airport，Harbourfront West，King and Spadina，Railway Lands，South Niagara,17,17,17,17,17,17
Cabbagetown，St. James Town,45,45,45,45,45,45
Central Bay Street,80,80,80,80,80,80
Chinatown，Grange Park，Kensington Market,92,92,92,92,92,92
Christie,16,16,16,16,16,16


In [43]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


In [44]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
Toronto_onehot.shape

(1685, 229)

In [46]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide，King，Richmond,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
2,Brockton，Exhibition Place，Parkdale Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,CN Tower，Bathurst Quay，Island airport，Harbourf...,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Cabbagetown，St. James Town,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,...,0.000000,0.000000,0.000000,0.00000,0.00,0.012500,0.000000,0.000000,0.012500,0.000000
7,Chinatown，Grange Park，Kensington Market,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.032609,0.000000,0.054348,0.010870,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.011494,0.011494,0.000000,0.00000,0.00,0.000000,0.011494,0.011494,0.000000,0.011494


In [47]:
Toronto_grouped.shape

(38, 229)

In [48]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide，King，Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2               Bar  0.04
3        Steakhouse  0.03
4  Asian Restaurant  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.04
2   Steakhouse  0.04
3  Cheese Shop  0.04
4       Bakery  0.04


----Brockton，Exhibition Place，Parkdale Village----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.05
2                   Spa  0.05
3  Gym / Fitness Center  0.05
4         Garden Center  0.05


----CN Tower，Bathurst Quay，Island airport，Harbourfront West，King and Spadina，Railway Lands，South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / 

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide，King，Richmond,Coffee Shop,Café,Bar,Cosmetics Shop,Bakery,Restaurant,Steakhouse,Sushi Restaurant,Asian Restaurant,Thai Restaurant
1,Berczy Park,Coffee Shop,Seafood Restaurant,Beer Bar,Bakery,Cocktail Bar,Steakhouse,Farmers Market,Cheese Shop,Café,Restaurant
2,Brockton，Exhibition Place，Parkdale Village,Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Climbing Gym,Caribbean Restaurant,Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
4,CN Tower，Bathurst Quay，Island airport，Harbourf...,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Coffee Shop,Plane,Boutique,Sculpture Garden,Bar,Airport Gate
5,Cabbagetown，St. James Town,Coffee Shop,Café,Bakery,Pizza Place,Pub,Market,Italian Restaurant,Restaurant,Bar,Playground
6,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Sandwich Place,Burger Joint,Japanese Restaurant,Spa,Salad Place,Gym / Fitness Center
7,Chinatown，Grange Park，Kensington Market,Café,Vietnamese Restaurant,Bar,Coffee Shop,Chinese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dumpling Restaurant,Farmers Market
8,Christie,Café,Grocery Store,Park,Restaurant,Italian Restaurant,Candy Store,Diner,Nightclub,Coffee Shop,Convenience Store
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Italian Restaurant,Gym,Mediterranean Restaurant,Men's Store


In [87]:
dfToronto.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
dfToronto.set_index('Neighborhood', drop = True, inplace = True)
dfToronto

,Postcode,Borough,Latitude,Longitude
Neighborhood,,,,
The Beaches,M4E,[East Toronto],43.676357,-79.293031
The Danforth West，Riverdale,M4K,[East Toronto],43.679557,-79.352188
The Beaches West，India Bazaar,M4L,[East Toronto],43.668999,-79.315572
Studio District,M4M,[East Toronto],43.659526,-79.340923
Lawrence Park,M4N,[Central Toronto],43.728020,-79.388790
Davisville North,M4P,[Central Toronto],43.712751,-79.390197
North Toronto West,M4R,[Central Toronto],43.715383,-79.405678
Davisville,M4S,[Central Toronto],43.704324,-79.388790
Moore Park，Summerhill East,M4T,[Central Toronto],43.689574,-79.383160


In [74]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = dfToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.reset_index(inplace  = True)
Toronto_merged.head() # check the last columns!

,Neighborhood,Postcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,M4E,[East Toronto],43.676357,-79.293031,0,Health Food Store,Trail,Pub,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,The Danforth West，Riverdale,M4K,[East Toronto],43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Caribbean Restaurant,Bakery,Indian Restaurant,Pub
2,The Beaches West，India Bazaar,M4L,[East Toronto],43.668999,-79.315572,0,Sushi Restaurant,Burger Joint,Ice Cream Shop,Liquor Store,Fish & Chips Shop,Pub,Italian Restaurant,Fast Food Restaurant,Burrito Place,Movie Theater
3,Studio District,M4M,[East Toronto],43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant,Martial Arts Dojo
4,Lawrence Park,M4N,[Central Toronto],43.728020,-79.388790,4,Dim Sum Restaurant,Park,Swim School,Bus Line,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [93]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.669542,-79.422564], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [98]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,0,Health Food Store,Trail,Pub,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Caribbean Restaurant,Bakery,Indian Restaurant,Pub
2,M4L,0,Sushi Restaurant,Burger Joint,Ice Cream Shop,Liquor Store,Fish & Chips Shop,Pub,Italian Restaurant,Fast Food Restaurant,Burrito Place,Movie Theater
3,M4M,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Diner,Park,Middle Eastern Restaurant,Martial Arts Dojo
5,M4P,0,Gym,Hotel,Convenience Store,Clothing Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Entertainment,Department Store
6,M4R,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Mexican Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Pet Store,Park
7,M4S,0,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Gym,Toy / Game Store,Restaurant
9,M4V,0,Pub,Coffee Shop,Light Rail Station,Bagel Shop,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Restaurant,American Restaurant
11,M4X,0,Coffee Shop,Café,Bakery,Pizza Place,Pub,Market,Italian Restaurant,Restaurant,Bar,Playground
12,M4Y,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Italian Restaurant,Gym,Mediterranean Restaurant,Men's Store


In [99]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,1,Playground,Trail,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,M4W,1,Park,Playground,Trail,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [100]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,2,Garden,Ice Cream Shop,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [101]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,3,Trail,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [102]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,4,Dim Sum Restaurant,Park,Swim School,Bus Line,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
